In [ ]:
import os,sys
%matplotlib inline
import random
import pandas as pd
import numpy as np
import pingouin
from itertools import cycle, islice
from pingouin import mediation_analysis
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from random import randint
from statannot import add_stat_annotation
from statsmodels.tools import add_constant
from statsmodels.regression.linear_model import OLS
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from scipy import stats 
import networkx as nx

import warnings
warnings.filterwarnings(action='ignore')

print(matplotlib.__version__)
print(sns.__version__)
print(pingouin.__version__)

# Data import

In [ ]:
PPGR_meal_merged = pd.read_csv('../../data/PPGR_meal_merged_cont2.csv',index_col=0) #Merged data (meal + CGM)
OTU_data = pd.read_csv('../../data/T2D_KBSMC_otu_norm.csv',index_col=0)             #16s microbiome data (taxonomic mapped read count profile)
med_data = pd.read_excel('../../data/CGM Nutrition_CRF_Medication.xlsx')            #Medication usage
clinical_data = pd.read_excel('../../data/CGM Nutrition_CRF_20230808_추가Lab.xlsx') #Clinicodemograhic data

In [ ]:
# Calculating PPGR #
CGM_post60_col =  [f"p_{t:03d}" for t in range(0,61,5)]
CGM_post120_col = [f"p_{t:03d}" for t in range(0,121,5)]
CGM_post240_col = [f"p_{t:03d}" for t in range(0,241,5)]

PPGR_meal_merged[CGM_post240_col] = PPGR_meal_merged.loc[:,CGM_post240_col].interpolate(axis=1) # Fill the Nan with interpolated value

G0 = PPGR_meal_merged['p_000']
# 2h PPGR #
iAUC_2h = [0]*len(PPGR_meal_merged)
for i in range(1,25):
    Gi = PPGR_meal_merged[CGM_post120_col[i]]
    Gi_1 = PPGR_meal_merged[CGM_post120_col[i-1]]
    Si = ((Gi-G0)+(Gi_1-G0))/2
    iAUC_2h+=Si*5

# 4h PPGR #
iAUC_4h = [0]*len(PPGR_meal_merged)
for i in range(1,49):
    Gi = PPGR_meal_merged[CGM_post240_col[i]]
    Gi_1 = PPGR_meal_merged[CGM_post240_col[i-1]]
    Si = ((Gi-G0)+(Gi_1-G0))/2
    iAUC_4h+=Si*5
    
PPGR_meal_merged['PPGR_u2'] = iAUC_2h/60
PPGR_meal_merged['PPGR_u4'] = iAUC_4h/60

In [ ]:
PPGR_meal_merged_filt = PPGR_meal_merged.dropna(subset=['meal_m1'],axis=0)
PPGR_meal_merged_filt = PPGR_meal_merged_filt[(PPGR_meal_merged['Energy(kcal)']<2000) & (PPGR_meal_merged['Carb(g)']<250)]
PPGR_meal_merged_filt['Carb_root'] = PPGR_meal_merged_filt['Carb(g)']**(1/2)
PPGR_meal_merged_filt['Carb_pro'] = PPGR_meal_merged_filt['Carb(g)']*4 / (PPGR_meal_merged_filt['Carb(g)']*4+PPGR_meal_merged_filt['Protein(g)']*4+PPGR_meal_merged_filt['Fat(g)']*9) * 100
PPGR_meal_merged_filt['Protein_pro'] = PPGR_meal_merged_filt['Protein(g)']*4 / (PPGR_meal_merged_filt['Carb(g)']*4+PPGR_meal_merged_filt['Protein(g)']*4+PPGR_meal_merged_filt['Fat(g)']*9) * 100
PPGR_meal_merged_filt['Fat_pro'] = PPGR_meal_merged_filt['Fat(g)']*9 / (PPGR_meal_merged_filt['Carb(g)']*4+PPGR_meal_merged_filt['Protein(g)']*4+PPGR_meal_merged_filt['Fat(g)']*9) * 100

OTU_data = OTU_data.loc[:,list(np.sum(OTU_data>0) > len(OTU_data)*0.3)]
microbiome_col = list(OTU_data.columns)
OTU_data_bc = pd.DataFrame(stats.boxcox(OTU_data,lmbda=0.25),columns=OTU_data.columns,index=OTU_data.index)

# Features
CGM_pre30_col = [f"m_{t:03d}" for t in range(30,0,-5)]+['p_000']
futureCGM_120 = [f"p_{t:03d}" for t in range(5,121,5)]
meal_composition = ['Carb(g)','Protein(g)','Fat(g)','Fiber(g)']
meal_context = ['meal_m1','prot_b6h','Time']
cli_col = ['Age','DM_Duration','BMI','HbA1c','SBP','DBP','HDL','LDL','AST(IU/L)','ALT(IU/L)'] 
med_col = ['Basal_Ins_Dose_Unit','MFM_Dose_Total','SU_Dose_Total','DPP4i_Dose_Total','SGLT2i_Dose_Total'] 

# Target features (2h PPGR & 4h PPGR)
PPGR = 'PPGR_u2'
PPGR_4 = 'PPGR_u4'

# Final dataframe
PPGR_data =  PPGR_meal_merged_filt.dropna(subset=['PPGR_u2','PPGR_u4']+CGM_pre30_col,axis=0)
full_data = pd.merge(PPGR_data[['ID','Dname',PPGR,PPGR_4,'Carb(g)','Meal']+CGM_pre30_col+CGM_post240_col+meal_composition+meal_context],OTU_data_bc[microbiome_col],left_on='ID',right_index=True)
full_data = full_data.T.drop_duplicates().T
full_data = pd.merge(full_data,med_data,how='left',on='ID')
full_data = pd.merge(full_data,clinical_data,how='left',on='ID')
full_data[PPGR] = full_data[PPGR].astype(float)
full_data[PPGR_4] = full_data[PPGR_4].astype(float)

samples = list(full_data['ID'].unique())
full_data

In [ ]:
evaluation_data = pd.read_csv('../..//data/validation/PPGR_meal_merged_validation.csv',index_col=0)
evaluation_data =  evaluation_data.dropna(subset=CGM_pre30_col,axis=0)
evaluation_data

# With 'pingouin' package

In [ ]:
causal_inference_df = pd.DataFrame(columns=['X','M','b2','b3','p2','total_b','direct_b','direct_p','indirect_b','indirect_p','mediation_prop'])
for nutrient in meal_composition:
    for species in microbiome_col:
        scaler = StandardScaler()

        full_data[species] = scaler.fit_transform(full_data[species].astype(float).values.reshape(-1, 1))
        full_data[nutrient] = scaler.fit_transform(full_data[nutrient].astype(float).values.reshape(-1, 1))

        mediation_df = mediation_analysis(data=full_data, x=nutrient, m=species,
                                    y='PPGR_u2', covar=None, alpha=0.05, n_boot=100, seed=2, return_dist=False, logreg_kwargs=None)
        
        p2 = mediation_df.loc[0,'pval']
        b2 = mediation_df.loc[0,'coef']
        p3 = mediation_df.loc[1,'pval']
        b3 = mediation_df.loc[1,'coef']
        total_b = mediation_df.loc[2,'coef']
        direct_b = mediation_df.loc[3,'coef']
        direct_p = mediation_df.loc[3,'pval']
        indirect_b = mediation_df.loc[4,'coef']
        indirect_p = mediation_df.loc[4,'pval']
        mediation_prop = mediation_df.loc[4,'coef']/mediation_df.loc[2,'coef']
        
        if  (p2<0.05) and (p3<0.05) and (indirect_p<0.05):
            res = {'b2':b2,'b3':b3,'p2':p2,'p3':p3,'total_b':total_b,
                   'direct_b':direct_b,'direct_p':direct_p,
                   'indirect_b':indirect_b,'indirect_p':indirect_p,
                   'mediation_prop':mediation_prop}
            res['X'] = nutrient
            res['M'] = species
            causal_inference_df = pd.concat([causal_inference_df,pd.DataFrame(res,index=[0])], ignore_index=True)

In [ ]:
causal_inference_df['M'] = ['\n'.join(var.split(';')[-2:]) for var in causal_inference_df['M']]
causal_inference_df

# Visualize network

In [ ]:
G = nx.DiGraph()
Mediators = list(causal_inference_df['M'].unique())
for _, row in causal_inference_df.iterrows():
    # X -> M edge
    G.add_edge(row['X'], row['M'], p_value=row['p2'], color='red' if row['b2'] > 0 else 'blue', alpha=0.5, weight=-np.log10(row['p2']))
    # M -> Y edge
    G.add_edge(row['M'], 'PPGR',  p_value=row['p3'], color='red' if row['b3'] > 0 else 'blue', alpha=0.5,  weight=-np.log10(row['p3']))

pos = {'Carb(g)':[-1, 3], 'Protein(g)':[-1, 1], 'Fat(g)':[-1, -1], 'Fiber(g)':[-1, -3], 
       'PPGR':[1, 0]
      }
for i,m in enumerate(Mediators):
    pos[m] = [0, 5-i]

edges = G.edges(data=True)
colors = [G[u][v]['color'] for u,v,_ in edges]
weights = [np.abs(G[u][v]['weight']) for u,v,_ in edges]

plt.figure(figsize=(8, 9))
plt.ylim(-6,6)
plt.xlim(-1.5, 1.5)

nx.draw_networkx_nodes(G, pos, node_color='white', node_size=4000,alpha=0)
nx.draw_networkx_labels(G, pos,)
nx.draw_networkx_edges(G, pos, arrowstyle='-|>', arrowsize=10, edge_color=colors, width=weights, 
                        min_source_margin=20,min_target_margin=40)

ax = plt.gca()

# Apply alpha to the arrows manually
for i, (u, v, d) in enumerate(G.edges(data=True)):
    arrow = ax.patches[i]
    arrow.set_alpha(0.4)
plt.axis('off')
plt.tight_layout()
plt.savefig('./figure/microbiome_causal_inference.jpeg',dpi=300)
plt.show()